this script investigate the impact of different funding type. Impact is measured by normalized citation

In [1]:
import pandas as pd
from ast import literal_eval
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from scipy.spatial import distance
from itertools import product
from adjustText import adjust_text
from matplotlib.patches import Patch

In [2]:
pubs_path="../../data/nf_folder/Data/DerivedData/CleanedRawData/pubs_author_funder.csv"
eufile_path='../../data/nf_folder/Data/AdditionalData/cntry_convert.xlsx'
cntry_cnt_path='../../data/nf_folder/Data/DerivedData/Derived/cntry_year_cnt.csv'

In [25]:
cntry_coll_fund_output='../../data/nf_folder/Data/DerivedData/Derived/coll_fund/cntry_coll_fund.csv'

In [4]:
pubs_author_funder = pd.read_csv(pubs_path)
pubs_author_funder['author_country'] = pubs_author_funder['author_country'].apply(literal_eval)
pubs_author_funder['funder_country'] = pubs_author_funder['funder_country'].apply(
    lambda x:literal_eval(x) if pd.notnull(x) else "Not-Funded")
pubs_author_funder.head()

/home/miaoli/conda/envs/funding/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,id,author_country,funder_country,discipline,year,rel_citation,citation
0,30507292,[Turkey],[Turkey],77,2009,0.485,6
1,30574690,"[United States, France]","[United States, United States, United States, ...",83,2009,0.703,9
2,30574691,[Russia],[Russia],83,2009,0.312,4
3,30585095,"[Spain, Spain]","[Spain, Spain, Spain, Spain, Spain, Spain]",89,2009,0.137,1
4,30585097,"[Germany, Spain]",[Spain],89,2009,0.410,3


In [5]:
eu_df=pd.read_excel(eufile_path)
eulist=eu_df[eu_df.EU==1].Country.unique()
eu_df.head()

<ipython-input-5-f1e2d5acec49>:1: FutureWarning: Your version of xlrd is 1.2.0. In xlrd >= 2.0, only the xls format is supported. As a result, the openpyxl engine will be used if it is installed and the engine argument is not specified. Install openpyxl instead.
  eu_df=pd.read_excel(eufile_path)


,Clarivate country,Country,Continent,EU
0,*EFFACER,NaN,NaN,NaN
1,_indeterminé,_indeterminé,NaN,NaN
2,AFGHANISTAN,Afghanistan,Asia,NaN
3,SOUTH-AFRICA,South Africa,Africa,NaN
4,ALBANIA,Albania,Europe,NaN


In [6]:
pubs_author_funder['author_distinct']=pubs_author_funder['author_country'].apply(lambda x:list(set(x)))
pubs_author_funder['funder_distinct']=pubs_author_funder['funder_country'].apply(
    lambda x:list(set(x)) if x!='Not-Funded' else 'Not-Funded')

In [7]:
pubs_author_funder['IntCol']=pubs_author_funder['author_distinct'].apply(
    lambda x: 'col' if len(x)>1 else 'notcol')

In [8]:
pubs_author_funder.head()

,id,author_country,funder_country,discipline,year,rel_citation,citation,author_distinct,funder_distinct,IntCol
0,30507292,[Turkey],[Turkey],77,2009,0.485,6,[Turkey],[Turkey],notcol
1,30574690,"[United States, France]","[United States, United States, United States, ...",83,2009,0.703,9,"[United States, France]",[United States],col
2,30574691,[Russia],[Russia],83,2009,0.312,4,[Russia],[Russia],notcol
3,30585095,"[Spain, Spain]","[Spain, Spain, Spain, Spain, Spain, Spain]",89,2009,0.137,1,[Spain],[Spain],notcol
4,30585097,"[Germany, Spain]",[Spain],89,2009,0.410,3,"[Germany, Spain]",[Spain],col


In [9]:
cntry_pub=pubs_author_funder[['id','author_distinct','funder_distinct','year','rel_citation','IntCol']
                            ].explode('author_distinct')
cntry_pub.head()

,id,author_distinct,funder_distinct,year,rel_citation,IntCol
0,30507292,Turkey,[Turkey],2009,0.485,notcol
1,30574690,United States,[United States],2009,0.703,col
1,30574690,France,[United States],2009,0.703,col
2,30574691,Russia,[Russia],2009,0.312,notcol
3,30585095,Spain,[Spain],2009,0.137,notcol


In [11]:
cntry_pub.head()

,id,author_distinct,funder_distinct,year,rel_citation,IntCol
0,30507292,Turkey,[Turkey],2009,0.485,notcol
1,30574690,United States,[United States],2009,0.703,col
1,30574690,France,[United States],2009,0.703,col
2,30574691,Russia,[Russia],2009,0.312,notcol
3,30585095,Spain,[Spain],2009,0.137,notcol


In [12]:
def check_fund(cntry_author,funderlist):
    label = 'null'
    if funderlist=='Not-Funded':
        label='not-identified'
    elif (cntry_author in eulist)&("EU" in funderlist):
        label='cofund'
    elif (cntry_author in funderlist)&(len(funderlist)==1):
        label='single'
    elif (cntry_author in funderlist)&(len(funderlist)>1):
        label='cofund'
    else:
        label='others'
    return label
cntry_pub['fund'] = cntry_pub.apply(lambda x:check_fund(x.author_distinct,x.funder_distinct),axis=1)
cntry_pub.head()

,id,author_distinct,funder_distinct,year,rel_citation,IntCol,fund
0,30507292,Turkey,[Turkey],2009,0.485,notcol,single
1,30574690,United States,[United States],2009,0.703,col,single
1,30574690,France,[United States],2009,0.703,col,others
2,30574691,Russia,[Russia],2009,0.312,notcol,single
3,30585095,Spain,[Spain],2009,0.137,notcol,single


In [26]:
cntry_pub.to_csv(cntry_coll_fund_output, index=False)